In [ ]:
print("test")

VBox()

In [14]:
csvDf = spark.read.csv("s3a://airbnb-spark/data/*.csv.gz", header="true", inferSchema="true", multiLine="true", escape='"')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
csvDf.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_pr

In [16]:
columnsToKeep = [
  "Metro",
  "price"]

baseDF = csvDf.select(columnsToKeep)
baseDF.cache().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

225926

In [17]:
from pyspark.sql.functions import col, translate

fixedPriceDF = baseDF.withColumn("price", translate(col("price"), "$,", "").cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
fixedPriceDF.select('Metro').distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Metro='Hawaii'), Row(Metro='Miami_USA'), Row(Metro='Santa_Cruz'), Row(Metro=None), Row(Metro='Austin'), Row(Metro='Rhode_Island'), Row(Metro='Chicago'), Row(Metro='Asheville'), Row(Metro='Salem'), Row(Metro='Washington_DC'), Row(Metro='Seattle'), Row(Metro='Las_Vegas'), Row(Metro='Twin_Cities'), Row(Metro='Pacific_Grove'), Row(Metro='San_Diego'), Row(Metro='Jersey_City'), Row(Metro='New_Orleans'), Row(Metro='Denver')]

In [36]:
from pyspark.sql.functions import lit

metropricedf=fixedPriceDF.groupby("Metro").avg("price")
metropricedf=metropricedf.withColumnRenamed("avg(price)","average_price")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
metropricedf=metropricedf.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
metropricedf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+
|        Metro|     average_price|
+-------------+------------------+
|       Hawaii| 353.9365072747738|
|    Miami_USA|204.30294805802527|
|   Santa_Cruz| 259.0538990825688|
|       Austin|454.65525456631076|
| Rhode_Island| 278.4712055285385|
|      Chicago|186.87369968340118|
|    Asheville|198.29951690821255|
|        Salem|236.64928909952607|
|Washington_DC|204.03716106730042|
|      Seattle|173.36268903291398|
|    Las_Vegas|202.69995431703975|
|  Twin_Cities| 504.3146883626326|
|Pacific_Grove| 416.4537037037037|
|    San_Diego| 257.0639429029204|
|  Jersey_City|  154.222333000997|
|  New_Orleans|235.55108679580067|
|       Denver| 178.6940329218107|
+-------------+------------------+

In [39]:
outputPath = "s3a://airbnb-spark/output"

metropricedf.write.mode("overwrite").parquet(outputPath)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…